# Analyse Stores in Transactions

In [1]:
import pandas as pd

stores = pd.read_csv("data/stores.csv")
transactions = pd.read_parquet("data/transactions_train_1.parquet")


In [ ]:

print(stores.columns) 

Index(['id', 'opening_date', 'location', 'state', 'urbanization',
       'sco_introduction'],
      dtype='object')


In [3]:
print("🔹 Erste Zeilen des Datensatzes:")
display(transactions.head())

🔹 Erste Zeilen des Datensatzes:


,id,store_id,cash_desk,transaction_start,transaction_end,total_amount,n_lines,payment_medium,customer_feedback,damage,label
0,5bedf771-f016-41a7-ab89-f40e7f6be955,b0973ffa-ce15-41e0-9de7-1390598e24f2,0,2022-02-02 08:00:12,2022-02-02 08:01:02.825563,52.26,6,CREDIT_CARD,NaN,NaN,UNKNOWN
1,21ff1406-dbc4-41a7-8429-836cbf96d853,b0973ffa-ce15-41e0-9de7-1390598e24f2,1,2022-02-02 08:05:06,2022-02-02 08:05:30.565876,1.47,1,CREDIT_CARD,NaN,NaN,UNKNOWN
2,b642f9b6-43b5-413d-ad08-b4bf4b664b52,b0973ffa-ce15-41e0-9de7-1390598e24f2,2,2022-02-02 08:07:12,2022-02-02 08:11:29.434607,363.87,37,CREDIT_CARD,NaN,0.0,NORMAL
3,16aa01c9-9678-4c6d-a007-baa3429b9fb8,b0973ffa-ce15-41e0-9de7-1390598e24f2,3,2022-02-02 08:11:45,2022-02-02 08:12:07.881710,3.43,1,CREDIT_CARD,NaN,NaN,UNKNOWN
4,0add22e5-6794-4e95-9d25-a1cd6b6cd083,b0973ffa-ce15-41e0-9de7-1390598e24f2,0,2022-02-02 08:15:15,2022-02-02 08:19:30.611551,441.40,47,CREDIT_CARD,NaN,NaN,UNKNOWN


In [4]:

print(transactions.columns)

Index(['id', 'store_id', 'cash_desk', 'transaction_start', 'transaction_end',
       'total_amount', 'n_lines', 'payment_medium', 'customer_feedback',
       'damage', 'label'],
      dtype='object')


## Join stores und transactions

Schlüsselattributs zur besseren Lesbarkeit umbenennen

In [5]:
stores.rename(columns={"id": "store_id"}, inplace=True)

Join

In [6]:
joined_df = transactions.merge(stores, on="store_id", how="left")

Welche Stores tauchen tatsächlich in Transaktionen auf?

In [7]:
# Alle Store-IDs mit mindestens einer Transaktion
stores_with_tx = transactions["store_id"].unique()

# Anzahl der betroffenen Stores
print(f"Anzahl Stores mit Transaktionen: {len(stores_with_tx)}")


Anzahl Stores mit Transaktionen: 3


Plausibilität prüfen

In [8]:
# Stores ohne Transaktionen herausfiltern
stores_without_tx = stores[~stores["store_id"].isin(stores_with_tx)]

# Anzahl der betroffenen Stores
print(f"Anzahl Stores ohne Transaktionen: {len(stores_without_tx)}")

Anzahl Stores ohne Transaktionen: 15


Betreffende Stores anzeigen

In [9]:
# Auswahl der gewünschten Spalten aus den betroffenen Stores
store_info = stores[stores["store_id"].isin(stores_with_tx)][["store_id", "opening_date", "location"]]

print(store_info)

                               store_id opening_date location
2  b0973ffa-ce15-41e0-9de7-1390598e24f2   2005-06-02   Berlin
4  d3892a9e-8cbc-4237-be9c-211435dc92c0   2003-02-10  München
9  ed996b17-17d6-47e8-9262-d1d02b52cdb4   1994-09-19  München


Transactions ohne Store?

In [10]:
fehlende_stores = joined_df["opening_date"].isnull().sum()
print(f"Anzahl Transaktionen ohne zugeordneten Store: {fehlende_stores}")

Anzahl Transaktionen ohne zugeordneten Store: 0
